In [11]:
#install Nemo Agent Toolkit and langchain dependency
!pip install uv
!uv pip install --system nvidia-nat
!uv pip install --system "nvidia-nat[langchain]"

Using Python 3.12.1 environment at: /usr/local/python/3.12.1
Resolved 140 packages in 1.03s                                       
⠙ Preparing packages... (0/33)                                                  
⠙ Preparing packages... (0/33)------------------     0 B/340.35 KiB          
⠙ Preparing packages... (0/33)------------------     0 B/340.35 KiB          
cffi                 ------------------------------     0 B/214.43 KiB
⠙ Preparing packages... (0/33)------------------     0 B/340.35 KiB          
cffi                 ------------------------------     0 B/214.43 KiB
⠙ Preparing packages... (0/33)------------------     0 B/340.35 KiB          
cffi                 ------------------------------     0 B/214.43 KiB
⠙ Preparing packages... (0/33)------------------     0 B/340.35 KiB          
charset-normalizer   ------------------------------     0 B/149.87 KiB
cffi                 ------------------------------     0 B/214.43 KiB
⠙ Preparing packages... (0/33)-------------

In [13]:
# load env variables 
from dotenv import load_dotenv
import os
   
# Load environment variables from .env file
load_dotenv()
   
# Verify the key loaded
print("API key loaded:", "Yes" if os.getenv('NVIDIA_API_KEY') else "No")
print("Nvidia endpoint loaded:", "Yes" if os.getenv('NVIDIA_BASE_URL') else "No")

API key loaded: Yes
Nvidia endpoint loaded: Yes


In [14]:
%%writefile config.yml

llms:
  climate_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    base_url: $NVIDIA_BASE_URL
    api_key: $NVIDIA_API_KEY
    temperature: 0.7
    max_tokens: 2048

workflow:
  _type: chat_completion
  llm_name: climate_llm
  system_prompt: |
    You are a knowledgeable climate science assistant. You help users understand 
    climate data, weather patterns, and global temperature trends. Be accurate, 
    informative, and cite scientific consensus when appropriate.

Overwriting config.yml


In [15]:
# Simple question about climate basics
!nat run \
  --config_file config.yml \
  --input "What is the difference between weather and climate?"

2025-12-25 21:35:17 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'config.yml'
2025-12-25 21:35:17 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function register_chat_completion by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.

Configuration Summary:
--------------------
Workflow Type: chat_completion
Number of Functions: 0
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2025-12-25 21:35:22 - INFO     - nat.front_ends.console.console_front_end_plugin:102 - --------------------------------------------------
Workflow Result:
['Weather and climate are two related but distinct concepts in the field of meteorology.\n\n**Wea

In [17]:
# Question about global temperature trends
!nat run \
  --config_file config.yml \
  --input "How much has global average \
  temperature increased since pre-industrial times?"

2025-12-25 21:38:04 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'config.yml'
2025-12-25 21:38:05 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function register_chat_completion by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.

Configuration Summary:
--------------------
Workflow Type: chat_completion
Number of Functions: 0
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2025-12-25 21:38:16 - INFO     - nat.front_ends.console.console_front_end_plugin:102 - --------------------------------------------------
Workflow Result:
["According to the Intergovernmental Panel on Climate Change (IPCC), the global average surface t

In [18]:
# A data-specific question that would benefit from real data analysis
!nat run \
  --config_file config.yml \
  --input "What were the exact temperature anomalies \
  for the top 5 warmest countries in 2023?"

2025-12-25 21:38:48 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'config.yml'
2025-12-25 21:38:48 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function register_chat_completion by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.

Configuration Summary:
--------------------
Workflow Type: chat_completion
Number of Functions: 0
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2025-12-25 21:38:53 - INFO     - nat.front_ends.console.console_front_end_plugin:102 - --------------------------------------------------
Workflow Result:
["According to the National Oceanic and Atmospheric Administration (NOAA), the top 5 warmest coun

In [20]:
# start the local api server
# subprocess nice trick 

import subprocess
import time 

# Clean up old processes if you ran notebook before
!pkill -f "nat serve" 2>/dev/null || true
time.sleep(2)

In [22]:
nat_process = subprocess.Popen(
    ["nat","serve", "--config_file", "config.yml", "--host", "127.0.0.1"],
     stdout = subprocess.PIPE,
     stderr=subprocess.PIPE,
     text=True
     
)

time.sleep(25)

In [23]:
import requests
import json 

# Test the API endpoint
response = requests.post(
    "http://localhost:8000/v1/chat/completions",
    headers={"Content-Type": "application/json"},
    json={
        "messages": [
            {
                "role": "user",
                "content": "What causes El Niño and how does it affect global weather?"
            }
        ],
        "stream": False
    }
)

# Parse and display the response
if response.status_code == 200:
    result = response.json()
    print(result["choices"][0]["message"]["content"])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

El Niño is a complex weather phenomenon that occurs when there are changes in the atmospheric and oceanic conditions over the eastern Pacific Ocean, near the equator. It's characterized by warmer-than-average sea surface temperatures (SSTs) in the central and eastern Pacific, near the equator, which can have significant impacts on global weather patterns.

**Causes of El Niño:**

El Niño is triggered by a combination of natural climate variability and changes in the atmospheric and oceanic conditions. Some of the key factors that contribute to El Niño include:

1. **Trade winds:** Normally, trade winds blow from east to west along the equator in the Pacific, pushing warm water towards Asia. During an El Niño event, these trade winds weaken or even reverse direction, allowing warm water to flow back towards the Americas.
2. **Thermocline:** The thermocline is a layer of water in the ocean where the temperature changes rapidly with depth. During an El Niño event, the thermocline deepens,

In [26]:
from ui_manager import ui_manager

ui_manager.start()


🔍 Checking prerequisites...
✅ All prerequisites found
🔍 Checking NAT server on port 8000...
✅ NAT server detected on port 8000
⚠️  Port 3000 is already in use
✅ UI repository already exists
📦 Installing UI dependencies...
✅ Dependencies installed
🎨 Starting UI development server...
⏳ Waiting for service on port 3000...
✅ UI started successfully on port 3000


True